In [1]:
import sys
sys.path.append('../tools')

In [2]:
import timm
from train_utils import *
# import tools.train_utils as utils

ModuleNotFoundError: No module named 'tools'

In [ ]:
base_path = 'data'
# base_path = '../data'
image_path = base_path + '/train_images/'
saved_path = base_path + '/saved_models/image_model/'
model_name = 'fine_tune_5epoch_nfnetl0-bs-64-pre-mgrad-adaptconcat-embed-1024-m08-10'
model_file = saved_path + model_name +'.pth'

In [ ]:
train_df = utils.no_split(pd.read_csv(base_path + '/train.csv'))

In [ ]:
class CONF( utils.ConfigClass):
    bs = 64
    embedding_size = 1024

conf = CONF()

In [ ]:
class ArcFaceNet(nn.Module):
    def __init__(self, arch, embedding_size=512, classifier=None):
        super().__init__()
        self.body = create_body(arch, cut=-2)
        nf = num_features_model(nn.Sequential(*self.body.children()))      
        self.after_conv=nn.Sequential(
                AdaptiveConcatPool2d(),
                Flatten(),
                nn.Linear(nf*2,embedding_size),
                nn.BatchNorm1d(embedding_size)
            )       

        self.classifier = classifier if classifier is not None else None

    def forward(self, x):
        x = self.body(x)
        return self.after_conv(x)

In [ ]:
def load_image_model(fname, arch_type = 'eca_nfnet_l0', embedding_size=1024):
    state_dict = torch.load(fname)
    arch = timm.create_model(arch_type, pretrained = False)
    model = ArcFaceNet(arch=arch, embedding_size=embedding_size)
    model.load_state_dict(state_dict)
    return model

In [ ]:
# image_getter = ImageFileGetter(PATH)
model = load_image_model(model_file, embedding_size=conf.embedding_size)

In [ ]:
# %debug

In [ ]:
dls = utils.get_image_dls(train_df,image_path,train_df.columns.get_loc("is_valid"),bs=conf.bs,train=False)

In [ ]:

embeddings = utils.get_embeddings(model, dls).cpu()

In [ ]:
save_folder = base_path + '/saved_embeddings/image_embeddings/'
save_name = model_name + '.pth'

In [ ]:
torch.save(embeddings, save_folder + save_name)